In [ ]:
using Images
import SimpleDraw as SD
using BenchmarkTools
using Paint
using StaticArrays

In [ ]:
import Base.size, Base.getindex, Base.setindex!

mutable struct MyArr{Arr, Pix} <: AbstractMatrix{Bool}
    background::Arr
    target::Arr
    colour::Pix
    total::Float32
end

function loss(source::RGB{Float32}, target::RGB{Float32})::Float32
    dr::Float32 = source.r - target.r
    dg::Float32 = source.g - target.g
    db::Float32 = source.b - target.b
    return (dr * dr) + (dg * dg) + (db * db)
end

Base.size(arr::MyArr) = size(arr.background)
Base.getindex(::MyArr, ::Int) = 0
Base.getindex(::MyArr, ::Int, ::Int) = 0
# Base.setindex!(::MyArr, ::Bool, ::Int) = nothing
function Base.setindex!(arr::MyArr, val::Bool, i::Int, j::Int)
    @inbounds arr.total += loss(arr.colour, arr.target[i, j]) - loss(arr.background[i, j], arr.target[i, j])
end

In [ ]:
col = RGB{Float32}(1, 0, 1)
image = MyArr(zeros(RGB{Float32}, 200, 200), ones(RGB{Float32}, 200, 200), col, 0.0f0)

# create the shape
pts = [9.0, 5, 14, 19, 30, 4] ./ 32.0 .* 200.0
pts = trunc.(Int32, pts)
shape = SD.FilledTriangle(SD.Point(pts[1], pts[2]), SD.Point(pts[3], pts[4]), SD.Point(pts[5], pts[6]))
tri = Triangle(SVector{6, Float32}(pts ./ 200.0))

# we will draw on the boolean image with the "color" true
color = true

# draw the shape on image


image = MyArr(zeros(RGB{Float32}, 200, 200), ones(RGB{Float32}, 200, 200), RGB{Float32}(1, 0, 1), 0.0f0)
SD.draw!(image, shape, color)

println(image.total)
println(drawloss(image.target, image.background, tri, col, SELoss(), RasterAlgorithmScanline()))

In [ ]:
@btime SD.draw!($image, $shape, $color)

In [ ]:
@btime drawloss($image.target, $image.background, $tri, $col, $SELoss(), $RasterAlgorithmScanline())